<a href="https://colab.research.google.com/github/kevorkian-mano/Motor_Vehicle_Collisions_Project/blob/main/FinalDashBoardDala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash==2.15.0 dash-bootstrap-components pyngrok plotly pandas


In [ ]:
from pyngrok import ngrok

# paste your token:
ngrok.set_auth_token("35jlMIAQ9iGTyEofWeKlw7dJuPe_5X87UVU6JDUwEnvNKMmoc")


In [75]:
# app_nyc_crash_dashboard.py
import ast
import numpy as np
import pandas as pd
import dash
from dash import dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from pyngrok import ngrok # optional - for local tunneling
import re

df = pd.read_csv("merged_cleaned_dataset.csv", dtype=str) # load as strings to be safe

# Normalize and cast useful columns
# Keep original columns but create convenient working columns
# Some columns have spaces in names; use exact names from your message.
# Convert crash datetime to datetime (coerce errors)
df["CRASH_DATETIME"] = pd.to_datetime(df["CRASH_DATETIME"], errors="coerce")
# YEAR for slider and groupings
df["YEAR"] = df["CRASH_DATETIME"].dt.year

# Cast numeric injury/killed counts to numeric (safe)
num_cols = [
     "NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED",
     "NUMBER OF PEDESTRIANS INJURED", "NUMBER OF PEDESTRIANS KILLED",
     "NUMBER OF CYCLIST INJURED", "NUMBER OF CYCLIST KILLED",
     "NUMBER OF MOTORIST INJURED", "NUMBER OF MOTORIST KILLED"
]
for c in num_cols:
     if c in df.columns:
          df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)
     else:
          df[c] = 0

# Helpful aggregated numeric columns
df["TOTAL_INJURED"] = df[["NUMBER OF PERSONS INJURED",
                         "NUMBER OF PEDESTRIANS INJURED",
                         "NUMBER OF CYCLIST INJURED",
                         "NUMBER OF MOTORIST INJURED"]].sum(axis=1)
df["TOTAL_KILLED"] = df[["NUMBER OF PERSONS KILLED",
                         "NUMBER OF PEDESTRIANS KILLED",
                         "NUMBER OF CYCLIST KILLED",
                         "NUMBER OF MOTORIST KILLED"]].sum(axis=1)

# Safe borough column
if "BOROUGH" not in df.columns:
     df["BOROUGH"] = "Unknown"
df["BOROUGH"] = df["BOROUGH"].fillna("Unknown")

# FULL_ADDRESS fallback
if "FULL ADDRESS" not in df.columns:
     df["FULL ADDRESS"] = df.get("ON STREET NAME", "").fillna("") + ", " + df.get("BOROUGH", "")

# Latitude / Longitude as numeric
for coord in ("LATITUDE", "LONGITUDE"):
     if coord in df.columns:
          df[coord] = pd.to_numeric(df[coord], errors="coerce")
     else:
          df[coord] = np.nan

# Parse ALL_VEHICLE_TYPES (which may be a string representation of a list) and create a flattened column
def parse_vehicle_list(v):
     if pd.isna(v):
          return []
     # If it's already a Python list object (rare in CSV), handle
     if isinstance(v, list):
          return [str(x).strip() for x in v if str(x).strip()]
     s = str(v).strip()
     # Try literal_eval if it's like "['SUV/Station Wagon', 'Sedan']"
     try:
          parsed = ast.literal_eval(s)
          if isinstance(parsed, (list, tuple)):
               return [str(x).strip() for x in parsed if str(x).strip()]
     except Exception:
          # fallback: comma-separated
          parts = [p.strip() for p in s.split(",") if p.strip()]
          return parts
     return []

df["VEHICLE_TYPES_LIST"] = df.get("ALL_VEHICLE_TYPES", "").apply(parse_vehicle_list)

# Expand vehicle types per row into a flat list column for easier counting
all_vehicle_types_flat = [vt for sub in df["VEHICLE_TYPES_LIST"] for vt in sub]
vehicle_type_counts = pd.Series(all_vehicle_types_flat).value_counts()
# Top 10 vehicle types for charts / heatmap combos
TOP_VEHICLE_TYPES = vehicle_type_counts.head(10).index.tolist()

# Parse contributing factors (all contributing factors column may be a list or string)
def parse_factor_list(v):
     if pd.isna(v):
          return []
     if isinstance(v, list):
          return [str(x).strip() for x in v if str(x).strip()]
     s = str(v).strip()
     try:
          parsed = ast.literal_eval(s)
          if isinstance(parsed, (list, tuple)):
               return [str(x).strip() for x in parsed if str(x).strip()]
     except Exception:
          parts = [p.strip() for p in s.split(",") if p.strip()]
          return parts
     return []

# Try to handle both ALL_CONTRIBUTING_FACTORS and ALL_CONTRIBUTING_FACTORS_STR
if "ALL_CONTRIBUTING_FACTORS" in df.columns:
     df["FACTORS_LIST"] = df["ALL_CONTRIBUTING_FACTORS"].apply(parse_factor_list)
elif "ALL_CONTRIBUTING_FACTORS_STR" in df.columns:
     df["FACTORS_LIST"] = df["ALL_CONTRIBUTING_FACTORS_STR"].apply(parse_factor_list)
else:
     # fallback to specific columns if provided
     parts = []
     for i in range(1, 4):
          c = f"CONTRIBUTING FACTOR VEHICLE {i}"
          if c in df.columns:
               parts.append(df[c].fillna("").astype(str))
     if parts:
          df["FACTORS_LIST"] = (pd.Series([";".join(x) for x in zip(*parts)]) if parts else pd.Series([[]]*len(df))).apply(
               lambda s: parse_factor_list(s))
     else:
          df["FACTORS_LIST"] = [[] for _ in range(len(df))]

all_factors_flat = [f for sub in df["FACTORS_LIST"] for f in sub]
factor_counts = pd.Series(all_factors_flat).value_counts()
TOP_FACTORS = factor_counts.head(10).index.tolist()

# PERSON_TYPE (type of persons involved)
if "PERSON_TYPE" not in df.columns and "PERSON_TYPE" in df.columns:
     pass
# ensure PERSON_TYPE column exists
if "PERSON_TYPE" not in df.columns:
     if "PERSON_TYPE" in df.columns:
          df["PERSON_TYPE"] = df["PERSON_TYPE"]
     else:
          df["PERSON_TYPE"] = df.get("PERSON_TYPE", "Unknown").fillna("Unknown")

# POSITION_IN_VEHICLE_CLEAN is provided in dataset per your list, ensure it's present
if "POSITION_IN_VEHICLE_CLEAN" not in df.columns:
     df["POSITION_IN_VEHICLE_CLEAN"] = df.get("POSITION_IN_VEHICLE_CLEAN", "").fillna("Unknown")

# Ensure other person-related columns exist (for new plots)
for col in ["PERSON_AGE", "PERSON_SEX", "BODILY_INJURY", "SAFETY_EQUIPMENT", "EMOTIONAL_STATUS", "UNIQUE_ID", "EJECTION", "ZIP CODE", "PERSON_INJURY"]:
    if col not in df.columns:
        # Create a placeholder column if not found (assuming person-level data is in the merged set)
        if col == "UNIQUE_ID":
            df[col] = df.index + 1
        elif col == "PERSON_AGE":
            df[col] = pd.to_numeric(df.get(col, np.nan), errors='coerce').fillna(0).astype(int) # Coerce age to int, fill missing/bad with 0
        elif col in ["EJECTION", "ZIP CODE", "PERSON_INJURY"]:
             df[col] = df.get(col, "Unknown").fillna("Unknown")
        else:
            df[col] = df.get(col, "Unknown").fillna("Unknown")

# Ensure additional columns exist
for col in ["COMPLAINT", "VEHICLE TYPE CODE 1", "CONTRIBUTING FACTOR VEHICLE 1"]:
    if col not in df.columns:
        df[col] = "Unknown"

# Small helper to add jitter to lat/lon to separate overlapping points
def jitter_coords(series, scale=0.0006):
     # scale tuned for city-level jitter
     return series + np.random.normal(loc=0, scale=scale, size=series.shape)

# Define consistent borough colors
BOROUGH_COLORS = {
    'MANHATTAN': '#2ECC71',  # Green
    'BROOKLYN': '#E74C3C',   # Red
    'QUEENS': '#3498DB',     # Blue
    'BRONX': '#F39C12',      # Orange
    'STATEN ISLAND': '#9B59B6', # Purple
    'Unknown': '#95A5A6'     # Gray
}

# ------------------------------------------------------------------
# Create Enhanced Dash app layout
# ------------------------------------------------------------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server


# Year slider marks
min_year = int(df["YEAR"].min()) if not df["YEAR"].isna().all() else 2010
max_year = int(df["YEAR"].max()) if not df["YEAR"].isna().all() else pd.Timestamp.now().year
year_marks = {y: str(y) for y in range(min_year, max_year + 1)}

app.layout = dbc.Container([
    # Header with Summary
    dbc.Row([
        dbc.Col([
            html.H1("💥 NYC Crash Analysis Dashboard",
                   className="text-center mb-4",
                   style={'color': '#ffffff', 'fontWeight': 'bold', 'fontSize': '2.5rem'}),
            html.Div(id="summary_text",
                    className="alert text-center",
                    style={'fontSize': '18px', 'fontWeight': 'bold', 'backgroundColor': '#FF8DA1', 'color': 'white', 'border': 'none'})
        ])
    ], className="mb-4"),

    # Interactive Control Panel
    dbc.Card([
        dbc.CardHeader(
            html.H4("📊 Control Panel", className="mb-0", style={'color': '#ffffff'}),
            style={'backgroundColor': '#FF8DA1'}
        ),
        dbc.CardBody([
         # Year Range Slider - Full Width
            dbc.Row([
                dbc.Col([
                    html.Label("Year Range", style={'color': '#ffffff', 'fontWeight': 'bold', 'fontSize': '16px'}),
                    dcc.RangeSlider(
                        id="year_slider",
                        min=min_year,
                        max=max_year,
                        value=[min_year, max_year],
                        marks={y: {'label': str(y), 'style': {'color': '#ffffff'}} for y in range(min_year, max_year + 1)},
                        tooltip={"placement": "bottom", "always_visible": True},
                        step=1,
                        allowCross=False
                    ),
                ], width=12),
            ], className="mb-4"),

            # Filters Row 1
            dbc.Row([
                dbc.Col([
                    html.Label("Borough", style={'color': '#ffffff', 'fontWeight': 'bold'}),
                    dcc.Dropdown(
                        id="borough_filter",
                        options=[{"label": b, "value": b} for b in sorted(df["BOROUGH"].dropna().unique())],
                        multi=True,
                        placeholder="All Boroughs",
                        style={'backgroundColor': '#FFE6E6', 'border': '1px solid #FFB6C1'}
                    )
                ], width=3),
                dbc.Col([
                    html.Label("Vehicle Type", style={'color': '#ffffff', 'fontWeight': 'bold'}),
                    dcc.Dropdown(
                        id="vehicle_filter",
                        options=[{"label": v, "value": v}
                                for v in sorted({vt for sub in df["VEHICLE_TYPES_LIST"] for vt in sub})],
                        multi=True,
                        placeholder="All Vehicle Types",
                        style={'backgroundColor': '#FFE6E6', 'border': '1px solid #FFB6C1'}
                    )
                ], width=3),
                dbc.Col([
                    html.Label("Contributing Factor", style={'color': '#ffffff', 'fontWeight': 'bold'}),
                    dcc.Dropdown(
                        id="factor_filter",
                        options=[{"label": f, "value": f}
                                for f in sorted({f for sub in df["FACTORS_LIST"] for f in sub})],
                        multi=True,
                        placeholder="All Factors",
                        style={'backgroundColor': '#FFE6E6', 'border': '1px solid #FFB6C1'}
                    )
                ], width=3),
                dbc.Col([
                    html.Label("Person Type", style={'color': '#ffffff', 'fontWeight': 'bold'}),
                    dcc.Dropdown(
                        id="person_type_filter",
                        options=[{"label": v, "value": v} for v in sorted(df["PERSON_TYPE"].dropna().unique())],
                        multi=True,
                        placeholder="All Person Types",
                        style={'backgroundColor': '#FFE6E6', 'border': '1px solid #FFB6C1'}
                    )
                ], width=3),
            ], className="mb-3"),

            # Filters Row 2
            dbc.Row([
                dbc.Col([
                    html.Label("Injury Type", style={'color': '#ffffff', 'fontWeight': 'bold'}),
                    dcc.Dropdown(
                        id="injury_filter",
                        options=[{"label": i, "value": i} for i in sorted(df["PERSON_INJURY"].dropna().unique())],
                        multi=True,
                        placeholder="All Injury Types",
                        style={'backgroundColor': '#FFE6E6', 'border': '1px solid #FFB6C1'}
                    )
                ], width=8),
                dbc.Col([
                    html.Label("Clear Filters", style={'color': '#ffffff', 'fontWeight': 'bold'}),
                    dbc.Button("🗑️ Clear All Filters",
                              id="clear_filters_btn",
                              color="warning",
                              size="md",
                              className="w-100",
                              style={
                                  'backgroundColor': '#FF6B6B',
                                  'border': 'none',
                                  'fontWeight': 'bold',
                                  'color': 'white'
                              })
                ], width=4),
            ], className="mb-4"),

            # Search Section - Now in its own dedicated section
            dbc.Row([
                dbc.Col([
                    html.Label("🔍 Advanced Search", style={'color': '#ffffff', 'fontWeight': 'bold', 'fontSize': '16px'}),
                    dbc.Input(
                        id="search_input",
                        placeholder="Try: 'queens 2019 to 2022 bicycle crashes caused by driver inattention female'...",
                        type="text",
                        style={
                            'backgroundColor': '#FFE6E6',
                            'border': '2px solid #FF8DA1',
                            'color': '#333',
                            'fontSize': '14px',
                            'padding': '12px'
                        }
                    ),
                    dbc.FormText(
                        "Search by borough, year, vehicle type, contributing factors, gender, injury type, etc.",
                        style={'color': '#ffffff', 'fontWeight': 'bold'}
                    )
                ], width=12),
            ], className="mb-4"),

            # Update Button
            dbc.Row([
                dbc.Col([
                    dbc.Button("🔄 Update Dashboard",
                              id="generate_btn",
                              color="primary",
                              size="lg",
                              className="w-100",
                              style={'backgroundColor': '#FF8DA1', 'border': 'none', 'fontWeight': 'bold'})
                ], width=12),
            ]),
        ], style={'backgroundColor': '#add8e6'})
    ], className="mb-4", style={'border': '2px solid #FF8DA1'}),

       # Tabbed Interface for Organized Content - FIXED TAB STYLING
    dbc.Tabs([
        # Tab 1: Crash Overview & Geography
        dbc.Tab([
            html.Br(),
            # Crash Map - Full Width
            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("📍 Crash Locations Map", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="map_chart", style={'height': '500px'})
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=12),
            ], className="mb-4"),

            # Crash Trends - Full Width
            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("📈 Crash Trends Over Time", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="crashes_by_year")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=12),
            ], className="mb-4"),

            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("🏙️ Crashes by Borough", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="borough_chart")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=6),
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("💥 Injuries by Borough", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="injuries_by_borough")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=6),
            ], className="mb-4"),
        ], label="🗺️ Crash Geography", tab_id="tab-1"),

        # Tab 2: Vehicle & Factor Analysis
        dbc.Tab([
            html.Br(),
            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("🔧 Contributing Factors", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="crashes_by_factor")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=6),
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("🔥 Vehicle vs Factor Heatmap", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="vehicle_factor_chart")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=6),
            ], className="mb-4"),

            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("🏎️ Vehicle Type Trends", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="vehicle_trend_chart")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=12),
            ], className="mb-4"),
        ], label="🏎️ Vehicles & Factors", tab_id="tab-2"),

        # Tab 3: Person & Injury Analysis
        dbc.Tab([
            html.Br(),
            # First row with Safety Equipment and Injury Types side by side
            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("🛡️ Safety Equipment", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="safety_equipment")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=6),
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("🚑 Injury Types", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="injury_chart")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=6),
            ], className="mb-4"),

            # Second row with the other three charts
            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("🎭 Emotional State", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="emotional_state")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=4),
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("🚪 Ejection Status", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="ejection_chart")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=4),
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("💺 Position in Vehicle", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="position_chart")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=4),
            ], className="mb-4"),

            # Third row with remaining charts
            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("👥 Person Types Over Time", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="injuries_by_person_type")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=6),
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("📋 Top Complaints", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="complaint_chart")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=6),
            ], className="mb-4"),
        ], label="👥 People & Injuries", tab_id="tab-3"),

        # Tab 4: Demographics & Statistics
        dbc.Tab([

            html.Br(),
            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("📊 Age Distribution", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="age_distribution_hist")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=8),
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("🚻 Gender Distribution", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    dcc.Graph(id="gender_distribution")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=4),
            ], className="mb-4"),

            dbc.Row([
                dbc.Col(dbc.Card(dbc.CardBody([
                    html.H5("📈 Real-time Statistics", style={'color': '#2C3E50', 'fontWeight': 'bold'}),
                    html.Div(id="live_stats", className="text-center")
                ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=12),
            ], className="mb-4"),
        ], label="📈 Demographics", tab_id="tab-4"),
    ], id="tabs", active_tab="tab-1",
       style={'marginTop': '20px'},
       className="custom-tabs"),

    # Footer
    dbc.Row([
        dbc.Col([
            html.Hr(style={'borderColor': '#FF8DA1'}),
            html.P("NYC Crash Analysis Dashboard | Built with Dash & Plotly",
                  className="text-center",
                  style={'color': '#ffffff', 'fontWeight': 'bold'})
        ])
    ], className="mt-4")

], fluid=True, style={'backgroundColor': '#cee8f0', 'minHeight': '100vh', 'padding': '20px'})

# Add custom CSS for tab styling AND slider
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            /* SLIDER STYLING */
            .rc-slider-track {
                background-color: #fc94af !important;
            }
            .rc-slider-rail {
                background-color: #FFC0CB !important;
            }
            .rc-slider-handle {
                background-color: #fc94af !important;
                border: 2px solid white !important;
            }

            /* TAB STYLING */
            .custom-tabs .nav-link {
                background-color: #FF8DA1 !important;
                color: white !important;
                border: 1px solid #FF8DA1 !important;
                font-weight: bold !important;
                margin-right: 5px;
            }
            .custom-tabs .nav-link.active {
                background-color: white !important;
                color: #FF8DA1 !important;
                border: 1px solid #FF8DA1 !important;
                font-weight: bold !important;
            }
            .custom-tabs .nav-link:hover {
                background-color: #FF85A1 !important;
                color: white !important;
            }
            .custom-tabs .nav-link.active:hover {
                background-color: white !important;
                color: #FF8DA1 !important;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# -------------------------
# Helper: parse search query - ENHANCED VERSION
# -------------------------
def parse_search_query(q):
    q = (q or "").lower().strip()
    found = {}

    # Extract years and year ranges
    year_pattern = r'\b(20\d{2})\b'
    years_found = re.findall(year_pattern, q)
    if years_found:
        years = sorted([int(y) for y in years_found])
        if len(years) >= 2:
            # If multiple years found, assume it's a range
            found["year_range"] = [years[0], years[-1]]
        else:
            # Single year
            found["year"] = years[0]

    # Borough detection
    borough_keywords = {
        'manhattan': 'MANHATTAN',
        'brooklyn': 'BROOKLYN',
        'queens': 'QUEENS',
        'bronx': 'BRONX',
        'staten': 'STATEN ISLAND',
        'staten island': 'STATEN ISLAND'
    }
    for keyword, borough in borough_keywords.items():
        if keyword in q:
            found["borough"] = [borough]
            break

    # Vehicle type detection
    vehicle_keywords = {
        'suv': 'SUV/Station Wagon',
        'station wagon': 'SUV/Station Wagon',
        'sedan': 'Sedan',
        'bicycle': 'Bicycle',
        'bike': 'Bicycle',
        'ambulance': 'Ambulance',
        'bus': 'Bus',
        'motorcycle': 'Motorcycle',
        'motorcycle': 'Motorcycle',
        'pickup': 'Pickup Truck',
        'pickup truck': 'Pickup Truck',
        'taxi': 'Taxi',
        'truck': 'Truck/Commercial',
        'commercial': 'Truck/Commercial',
        'van': 'Van',
        'pedicab': 'Pedicab'
    }
    vehicle_matches = []
    for keyword, vehicle_type in vehicle_keywords.items():
        if keyword in q:
            vehicle_matches.append(vehicle_type)
    if vehicle_matches:
        found["vehicle"] = vehicle_matches

    # Contributing factor detection
    factor_keywords = {
        'tailgating': 'tailgating',
        'unsafe': 'unsafe maneuver',
        'failure to yield': 'failure to yield',
        'vehicle defect': 'vehicle defect',
        'traffic violation': 'traffic violation',
        'speeding': 'speeding',
        'visibility': 'visibility issue',
        'substance': 'substance impairment',
        'slippery': 'slippery pavement',
        'pedestrian error': 'pedestrian or bicyclist error',
        'bicyclist error': 'pedestrian or bicyclist error',
        'passenger distraction': 'passenger distraction',
        'fatigue': 'driver fatigue',
        'driver inattention': 'driver inattention',
        'driver inexperience': 'driver inexperience',
        'conscious': 'Apparent Death',  # Emotional state mapping
        'unconscious': 'Apparent Death'
    }
    factor_matches = []
    for keyword, factor in factor_keywords.items():
        if keyword in q:
            factor_matches.append(factor)
    if factor_matches:
        found["factor"] = factor_matches

    # Person type and demographics
    if 'female' in q or 'woman' in q:
        found["gender"] = ['F']
    if 'male' in q or 'man' in q:
        found["gender"] = ['M']
    if 'pedestrian' in q:
        found["person_type"] = ['Pedestrian']
    if 'cyclist' in q or 'bicyclist' in q:
        found["person_type"] = ['Cyclist']
    if 'motorist' in q:
        found["person_type"] = ['Motorist']
    if 'driver' in q:
        found["person_type"] = ['Driver']

    # Injury severity
    if 'injured' in q or 'injury' in q:
        found["injury"] = ['Injured']
    if 'killed' in q or 'fatal' in q or 'fatality' in q or 'death' in q:
        found["injury"] = ['Killed']

    return found

# -------------------------
# Enhanced Callback with Pink Plot Backgrounds
# -------------------------
@app.callback(
     [
          # Main Outputs
          Output("injuries_by_borough", "figure"),
          Output("crashes_by_factor", "figure"),
          Output("crashes_by_year", "figure"),
          Output("map_chart", "figure"),
          Output("gender_distribution", "figure"),
          Output("safety_equipment", "figure"),
          Output("emotional_state", "figure"),
          Output("age_distribution_hist", "figure"),
          Output("injuries_by_person_type", "figure"),
          Output("summary_text", "children"),

          # Dareen's Outputs
          Output("borough_chart", "figure"),
          Output("injury_chart", "figure"),
          Output("ejection_chart", "figure"),
          Output("complaint_chart", "figure"),
          Output("vehicle_factor_chart", "figure"),
          Output("position_chart", "figure"),
          Output("vehicle_trend_chart", "figure"),

          # Live Stats
          Output("live_stats", "children"),
     ],
     Input("generate_btn", "n_clicks"),
     [
          State("year_slider", "value"),
          State("borough_filter", "value"),
          State("vehicle_filter", "value"),
          State("factor_filter", "value"),
          State("injury_filter", "value"),
          State("person_type_filter", "value"),
          State("search_input", "value"),
     ]
)
def update_dashboard(n_clicks, year_range, boroughs, vehicles, factors, injuries, person_type, search_text):
     dff = df.copy()

     # --- Apply search query ---
     if search_text:
          parsed = parse_search_query(search_text)
          print(f"Parsed search: {parsed}")  # Debug line

          # Year range from search
          if "year_range" in parsed:
               yr_range = parsed["year_range"]
               year_range = [max(year_range[0], yr_range[0]), min(year_range[1], yr_range[1])]
          elif "year" in parsed:
               yr = parsed["year"]
               year_range = [max(year_range[0], yr), min(year_range[1], yr)]

          # Borough filter from search
          if "borough" in parsed:
               if boroughs:
                    boroughs = list(set(boroughs) & set(parsed["borough"]))
               else:
                    boroughs = parsed["borough"]

          # Vehicle filter from search
          if "vehicle" in parsed:
               if vehicles:
                    vehicles = list(set(vehicles) & set(parsed["vehicle"]))
               else:
                    vehicles = parsed["vehicle"]

          # Factor filter from search
          if "factor" in parsed:
               if factors:
                    factors = list(set(factors) & set(parsed["factor"]))
               else:
                    factors = parsed["factor"]

          # Person type filter from search
          if "person_type" in parsed:
               if person_type:
                    person_type = list(set(person_type) & set(parsed["person_type"]))
               else:
                    person_type = parsed["person_type"]

          # Injury filter from search
          if "injury" in parsed:
               if injuries:
                    injuries = list(set(injuries) & set(parsed["injury"]))
               else:
                    injuries = parsed["injury"]

          # Gender filter from search
          if "gender" in parsed:
               dff = dff[dff["PERSON_SEX"].isin(parsed["gender"])]

     # --- Apply year range ---
     if year_range and len(year_range) == 2:
          y0, y1 = int(year_range[0]), int(year_range[1])
          dff = dff[(dff["YEAR"] >= y0) & (dff["YEAR"] <= y1)]

     # --- Apply borough filter ---
     if boroughs:
          dff = dff[dff["BOROUGH"].isin(boroughs)]

     # --- Apply injury filter ---
     if injuries:
          dff = dff[dff["PERSON_INJURY"].fillna("").astype(str).isin([str(i) for i in injuries])]

     # --- Apply vehicle filter ---
     if vehicles:
          mask = dff["VEHICLE_TYPES_LIST"].apply(lambda lst: any(v in (lst if isinstance(lst, list) else []) for v in vehicles))
          dff = dff[mask]

     # --- Apply contributing factor filter ---
     if factors:
          # Clean the factors to remove any extra characters
          clean_factors = [str(f).strip().strip("[]'\"") for f in factors]
          mask = dff["FACTORS_LIST"].apply(lambda lst: any(
              any(clean_f in str(fact).strip().strip("[]'\"") for clean_f in clean_factors)
              for fact in (lst if isinstance(lst, list) else [])
          ))
          dff = dff[mask]

     # --- Apply person type filter ---
     if person_type:
          dff = dff[dff["PERSON_TYPE"].isin(person_type)]

     # Calculate statistics for live stats
     total_crashes = len(dff)
     total_injuries = dff["TOTAL_INJURED"].sum()
     total_killed = dff["TOTAL_KILLED"].sum()
     avg_injuries_per_crash = total_injuries / total_crashes if total_crashes > 0 else 0

     # Define pink plot template
     pink_template = {
         'layout': {
             'paper_bgcolor': '#FFE6E6',
             'plot_bgcolor': '#FFE6E6',
             'font': {'color': '#2C3E50'},
             'xaxis': {'gridcolor': '#FFB6C1', 'linecolor': '#2C3E50'},
             'yaxis': {'gridcolor': '#FFB6C1', 'linecolor': '#2C3E50'}
         }
     }

     # Vibrant color sequence for line charts
     vibrant_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7', '#DDA0DD', '#98D8C8', '#F7DC6F', '#BB8FCE', '#85C1E9']

     # ---------- LAMA'S FIGURES ----------
    # 1) Injuries by Borough - Consistent borough colors
     injuries_by_borough = dff.groupby("BOROUGH")["TOTAL_INJURED"].sum().reset_index().sort_values("TOTAL_INJURED", ascending=False)
     fig_inj_borough = px.bar(injuries_by_borough, x="BOROUGH", y="TOTAL_INJURED",
                              labels={"TOTAL_INJURED": "Total Injured", "BOROUGH": "Borough"},
                              text="TOTAL_INJURED",
                              color="BOROUGH",
                              color_discrete_map=BOROUGH_COLORS)
     fig_inj_borough.update_traces(textposition="outside")
     fig_inj_borough.update_layout(margin=dict(t=40, b=20), template=pink_template, showlegend=False)

     # 2) Crashes by Contributing Factor - Purple shades
     factor_rows = []
     for _, row in dff.iterrows():
          for f in row["FACTORS_LIST"]:
               factor_rows.append((f, row["UNIQUE_ID"] if "UNIQUE_ID" in row else 1))
     factor_df = pd.DataFrame(factor_rows, columns=["Factor", "UID"]) if factor_rows else pd.DataFrame(columns=["Factor", "UID"])
     factor_counts_df = factor_df["Factor"].value_counts().head(15).reset_index()
     factor_counts_df.columns = ["Factor", "Count"]
     fig_factor = px.bar(factor_counts_df, x="Count", y="Factor", orientation="h",
                         labels={"Count": "Number of Crashes", "Factor": "Contributing Factor"},
                         color="Count",
                         color_continuous_scale="purples")
     fig_factor.update_layout(margin=dict(t=40, b=20), yaxis={'categoryorder':'total ascending'}, template=pink_template)

     # 3) Crashes per Year (line) by Borough - Consistent borough colors
     year_group = dff.groupby(["YEAR", "BOROUGH"]).size().reset_index(name="Crashes")
     if not year_group.empty:
          fig_year = px.line(year_group, x="YEAR", y="Crashes", color="BOROUGH", markers=True,
                          color_discrete_map=BOROUGH_COLORS)
          fig_year.update_layout(template=pink_template)
     else:
          fig_year = go.Figure()
          fig_year.update_layout(title="Crashes per Year (no data for selection)")

     # 4) Crash locations points map - Consistent borough colors
     df_map = dff.dropna(subset=["LATITUDE", "LONGITUDE"]).copy()
     if not df_map.empty:
          df_map["_LAT_JIT"] = jitter_coords(df_map["LATITUDE"].fillna(0).astype(float), scale=0.0005)
          df_map["_LON_JIT"] = jitter_coords(df_map["LONGITUDE"].fillna(0).astype(float), scale=0.0005)
          fig_map = px.scatter_mapbox(df_map, lat="_LAT_JIT", lon="_LON_JIT", color="BOROUGH",
                                    hover_name="FULL ADDRESS",
                                    hover_data={"FULL ADDRESS": True,
                                                "TOTAL_INJURED": True,
                                                "TOTAL_KILLED": True,
                                                "CRASH_DATETIME": True,
                                                "_LAT_JIT": False, "_LON_JIT": False},
                                    zoom=9, height=500,
                                    mapbox_style="open-street-map",
                                    color_discrete_map=BOROUGH_COLORS)
          fig_map.update_traces(marker=dict(size=8, opacity=0.7))
          fig_map.update_layout(margin=dict(t=0), template=pink_template)
     else:
          fig_map = go.Figure()
          fig_map.update_layout(title="No location data to display")

     # 5) Gender Distribution
     gender_dist = dff.groupby("PERSON_SEX")["UNIQUE_ID"].count().reset_index(name="Count")
     fig_gender = px.pie(gender_dist, names="PERSON_SEX", values="Count",
                        color_discrete_sequence=['#4A90E2', '#FF8DA1', '#95A5A6'])
     fig_gender.update_layout(margin=dict(t=40, b=20), template=pink_template)

     # 6) Safety Equipment Usage (Top 5)
     safety_dist = dff.groupby("SAFETY_EQUIPMENT")["UNIQUE_ID"].count().reset_index(name="Count")
     safety_dist = safety_dist.sort_values("Count", ascending=False).head(5)
     fig_safety = px.pie(safety_dist, names="SAFETY_EQUIPMENT", values="Count",
                         labels={"SAFETY_EQUIPMENT": "Safety Equipment", "Count": "Number of Records"},
                         color_discrete_sequence=['#FF8DA1', '#FFB6C1', '#FFD1DC', '#FFAEC9', '#FF85A1'])
     fig_safety.update_layout(margin=dict(t=40, b=20), template=pink_template)

     # 7) Emotional State Distribution - Pink color
     emotional_dist = dff.groupby("EMOTIONAL_STATUS")["UNIQUE_ID"].count().reset_index(name="Count")
     emotional_dist = emotional_dist.sort_values("Count", ascending=False)
     fig_emotional = px.bar(emotional_dist, x="EMOTIONAL_STATUS", y="Count",
                             labels={"EMOTIONAL_STATUS": "Emotional State", "Count": "Number of Records"},
                             color_discrete_sequence=['#FF8DA1'])
     fig_emotional.update_layout(margin=dict(t=40, b=20), xaxis={'categoryorder':'total descending'}, template=pink_template, showlegend=False)

     # 8) Age Distribution with Marginal Box Plot
     dff["PERSON_AGE"] = pd.to_numeric(dff["PERSON_AGE"], errors='coerce')
     fig_age_hist = px.histogram(dff, x="PERSON_AGE", nbins=30,
                           marginal="box",
                           hover_data=["PERSON_AGE"],
                           color_discrete_sequence=['#FF6B6B'])
     fig_age_hist.update_layout(
         margin=dict(t=40, b=20),
         xaxis_title="Age",
         yaxis_title="Count",
         template=pink_template
     )

     # 9) Injuries by Person Type Over Time
     person_type_time = dff.groupby(["YEAR", "PERSON_TYPE"]).agg({
         "TOTAL_INJURED": "sum"
     }).reset_index()
     fig_person_time = px.bar(person_type_time, x="YEAR", y="TOTAL_INJURED", color="PERSON_TYPE",
                             barmode="stack",
                             color_discrete_sequence=vibrant_colors)
     fig_person_time.update_layout(
         margin=dict(t=40, b=20),
         legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
         template=pink_template
     )

     # --- Summary text generation ---
     summary = f"📊 Currently showing: {total_crashes:,} crashes | {total_injuries:,} injured | {total_killed:,} fatalities"

     # ---------- DAREEN'S FIGURES ----------
     # 1. Borough chart - Consistent borough colors
     borough_df = dff.groupby("BOROUGH").size().reset_index(name="Count").sort_values("Count", ascending=False)
     fig_borough_dareen = px.bar(borough_df, x="BOROUGH", y="Count",
                              color="BOROUGH",
                              color_discrete_map=BOROUGH_COLORS)
     fig_borough_dareen.update_layout(template=pink_template, showlegend=False)

     # 2. Injury-Type chart (horizontal) - Teal color
     injury_df = dff.groupby("BODILY_INJURY").size().reset_index(name="Count").sort_values("Count", ascending=False)
     fig_injury_dareen = px.bar(
          injury_df,
          x="Count",
          y="BODILY_INJURY",
          orientation="h",
          labels={"BODILY_INJURY": "Bodily Injury", "Count": "Number of Cases"},
          color_discrete_sequence=['#20B2AA']  # Teal color
     )
     fig_injury_dareen.update_yaxes(categoryorder="total ascending")
     fig_injury_dareen.update_layout(margin=dict(t=40, b=20), template=pink_template, showlegend=False)

     # 3. Ejection chart (grouped by person type)
     fig_ejection = px.bar(
          dff.groupby(["PERSON_TYPE", "EJECTION"]).size().reset_index(name="Count"),
          x="EJECTION",
          y="Count",
          color="PERSON_TYPE",
          labels={"EJECTION": "Ejection Status", "Count": "Number of Cases"},
          color_discrete_sequence=vibrant_colors
     )
     fig_ejection.update_layout(template=pink_template)

     # 4. Complaint chart (grouped by person type)
     top_complaints = dff["COMPLAINT"].value_counts().nlargest(10).index
     fig_complaint = px.bar(
          dff[dff["COMPLAINT"].isin(top_complaints)].groupby(["COMPLAINT", "PERSON_TYPE"])
          .size().reset_index(name="Count"),
          x="COMPLAINT",
          y="Count",
          color="PERSON_TYPE",
          labels={"COMPLAINT": "Complaint Type", "Count": "Number of Cases"},
          color_discrete_sequence=vibrant_colors
     )
     fig_complaint.update_layout(xaxis_tickangle=-45, template=pink_template)

     # 5. Vehicle Factor Heatmap
     top_factors = dff["CONTRIBUTING FACTOR VEHICLE 1"].value_counts().nlargest(10).index
     fig_vehicle_factor = px.density_heatmap(
          dff[dff["CONTRIBUTING FACTOR VEHICLE 1"].isin(top_factors)],
          x="VEHICLE TYPE CODE 1",
          y="CONTRIBUTING FACTOR VEHICLE 1",
          labels={"VEHICLE TYPE CODE 1": "Vehicle Type", "CONTRIBUTING FACTOR VEHICLE 1": "Contributing Factor"},
          color_continuous_scale="viridis"
     )
     fig_vehicle_factor.update_layout(template=pink_template)

     # 6. Position chart
     fig_position = px.bar(
          dff.groupby(["POSITION_IN_VEHICLE_CLEAN", "PERSON_INJURY"]).size().reset_index(name="Count"),
          x="POSITION_IN_VEHICLE_CLEAN",
          y="Count",
          color="PERSON_INJURY",
          labels={"POSITION_IN_VEHICLE_CLEAN": "Position in Vehicle", "Count": "Number of Cases"},
          color_discrete_sequence=vibrant_colors
     )
     fig_position.update_layout(template=pink_template)

     # 7. Vehicle trend chart - Vibrant colors
     trend_df = dff.groupby(["YEAR", "VEHICLE TYPE CODE 1"]).size().reset_index(name="Count")
     fig_vehicle_trend = px.line(
          trend_df,
          x="YEAR",
          y="Count",
          color="VEHICLE TYPE CODE 1",
          labels={"YEAR": "Year", "Count": "Number of Crashes", "VEHICLE TYPE CODE 1": "Vehicle Type"},
          color_discrete_sequence=vibrant_colors
     )
     fig_vehicle_trend.update_layout(template=pink_template)

     # Live Statistics Display with matching colors
     live_stats = dbc.Row([
         dbc.Col(dbc.Card(dbc.CardBody([
             html.H4("🏎️ Total Crashes", className="text-primary", style={'color': '#FF8DA1'}),
             html.H2(f"{total_crashes:,}", style={'color': '#FF8DA1', 'fontWeight': 'bold'})
         ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=3),
         dbc.Col(dbc.Card(dbc.CardBody([
             html.H4("💥 Total Injuries", className="text-warning", style={'color': '#FF8DA1'}),
             html.H2(f"{total_injuries:,}", style={'color': '#FF8DA1', 'fontWeight': 'bold'})
         ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=3),
         dbc.Col(dbc.Card(dbc.CardBody([
             html.H4("💀 Total Fatalities", className="text-danger", style={'color': '#FF8DA1'}),
             html.H2(f"{total_killed:,}", style={'color': '#FF8DA1', 'fontWeight': 'bold'})
         ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=3),
         dbc.Col(dbc.Card(dbc.CardBody([
             html.H4("📈 Avg Injuries/Crash", className="text-success", style={'color': '#FF8DA1'}),
             html.H2(f"{avg_injuries_per_crash:.2f}", style={'color': '#FF8DA1', 'fontWeight': 'bold'})
         ]), style={'backgroundColor': '#FFE6E6', 'border': '2px solid #FF8DA1'}), md=3),
     ])

     return (
          # LAMA'S FIGURES
          fig_inj_borough,
          fig_factor,
          fig_year,
          fig_map,
          fig_gender,
          fig_safety,
          fig_emotional,
          fig_age_hist,
          fig_person_time,
          summary,

          # DAREEN'S FIGURES
          fig_borough_dareen,
          fig_injury_dareen,
          fig_ejection,
          fig_complaint,
          fig_vehicle_factor,
          fig_position,
          fig_vehicle_trend,

          # Live Stats
          live_stats,
     )

# Clear Filters Callback
@app.callback(
    [
        Output("year_slider", "value"),
        Output("borough_filter", "value"),
        Output("vehicle_filter", "value"),
        Output("factor_filter", "value"),
        Output("injury_filter", "value"),
        Output("person_type_filter", "value"),
        Output("search_input", "value")
    ],
    Input("clear_filters_btn", "n_clicks"),
    prevent_initial_call=True
)
def clear_all_filters(n_clicks):
    min_year = int(df["YEAR"].min()) if not df["YEAR"].isna().all() else 2010
    max_year = int(df["YEAR"].max()) if not df["YEAR"].isna().all() else pd.Timestamp.now().year

    return (
        [min_year, max_year],  # year_slider
        None,  # borough_filter
        None,  # vehicle_filter
        None,  # factor_filter
        None,  # injury_filter
        None,  # person_type_filter
        ""     # search_input
    )


In [76]:
ngrok.kill()
public_url = ngrok.connect(8050)
print("Dashboard running on:", public_url)

app.run_server(port=8050)


Dashboard running on: NgrokTunnel: "https://lajuana-uncopied-bessie.ngrok-free.dev" -> "http://localhost:8050"


<IPython.core.display.Javascript object>